In [152]:
import pandas as pd
import sklearn as skt
import re
from datetime import datetime, timezone
import matplotlib.pyplot as plt

In [ ]:
# Criar arquivo CSV
def baseArchive():
    basePath = os.path.join(os.getcwd(), 'db', 'FakeReal.csv')
    os.makedirs(os.path.dirname(basePath), exist_ok=True)
    if not os.path.exists(basePath):
        with open(basePath, 'w', encoding='utf-8') as baseArchive:
            baseArchive.write('titulo,URL\n')  # Cabeçalho compatível com os dados
    print(f"Arquivo base criado ou já existente em: {basePath}")
    return basePath

In [ ]:
def saveData(data, filename=None):
    if filename is None:
        filename = baseArchive()

    df = pd.DataFrame(data)

    try:
        # Caso o arquivo já exista
        if os.path.exists(filename):
            existing_df = pd.read_csv(filename)
            
            # Combina os novos dados com os existentes, sem remover duplicatas
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df.to_csv(filename, index=True, index_label='index')
        else:
            # Caso seja o primeiro salvamento
            df.to_csv(filename, index=True, index_label='index')

        print(f"Dados salvos no arquivo '{filename}' com sucesso!")

    except Exception as e:
        print(f"Erro ao salvar dados: {e}")

In [153]:
df = pd.read_csv('db/full_FatoFake.csv')
df.head()

,index,titulo,resumo,URL
0,0,É #FAKE foto que mostra Alexandre de Moraes us...,imagem criada com inteligência artificial most...,https://g1.globo.com/fato-ou-fake/noticia/2025...
1,1,Fato ou Fake estreia quadro no 'Mais Você'; as...,no programa apresentado por ana maria braga na...,https://g1.globo.com/fato-ou-fake/noticia/2025...
2,2,"Paula Paiva apresenta o quadro ""Fato ou Fake"" ...",paula paiva apresenta o quadro fato ou fake no...,https://g1.globo.com/fato-ou-fake/video/paula-...
3,3,É #FAKE que governo entregou administração de ...,é fake que governo entregou administração de d...,https://g1.globo.com/fato-ou-fake/video/e-fake...
4,4,#NÃO É BEM ASSIM: Nem todo acompanhante de pas...,posts acertam na descrição de como obter desco...,https://g1.globo.com/fato-ou-fake/noticia/2025...


In [154]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16983 entries, 0 to 16982
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   16983 non-null  int64 
 1   titulo  16983 non-null  object
 2   resumo  9990 non-null   object
 3   URL     16983 non-null  object
dtypes: int64(1), object(3)
memory usage: 530.8+ KB


In [155]:
# df = df.dropna(subset=['resumo'])
df['resumo'] = df['resumo'].fillna('não há dados sobre')


In [156]:
# Definir uma função para verificar se as frases específicas estão no 'resumo'
def remove_noticias_irrelevantes(resumo):
    return ('estreia quadro no "Mais Você"' not in resumo) and ('apresenta o quadro "Fato ou Fake"' not in resumo)


In [157]:
# Aplicar a função para filtrar as linhas
df = df[df['resumo'].apply(remove_noticias_irrelevantes)]

In [158]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16983 entries, 0 to 16982
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   16983 non-null  int64 
 1   titulo  16983 non-null  object
 2   resumo  16983 non-null  object
 3   URL     16983 non-null  object
dtypes: int64(1), object(3)
memory usage: 530.8+ KB


In [159]:
keys = {
    'fake': ['#FAKE', '#NÃO É BEM ASSIM']
    }

In [160]:
# Função para classificar
def classificar(row):
    # Verificar nas colunas 'titulo' e 'resumo'
    texto_completo = row['titulo'] + " " + row['resumo']  # Junta as duas colunas em uma string
    # Verifica se alguma palavra-chave de 'fake' aparece
    if any(palavra in texto_completo for palavra in keys['fake']):
        return 'fake'
    return 'fato'  # Se não encontrar 'fake', classifica como 'fato'


In [161]:
df['check'] = df.apply(classificar, axis=1)
df.head()


,index,titulo,resumo,URL,check
0,0,É #FAKE foto que mostra Alexandre de Moraes us...,imagem criada com inteligência artificial most...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fake
1,1,Fato ou Fake estreia quadro no 'Mais Você'; as...,no programa apresentado por ana maria braga na...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fato
2,2,"Paula Paiva apresenta o quadro ""Fato ou Fake"" ...",paula paiva apresenta o quadro fato ou fake no...,https://g1.globo.com/fato-ou-fake/video/paula-...,fato
3,3,É #FAKE que governo entregou administração de ...,é fake que governo entregou administração de d...,https://g1.globo.com/fato-ou-fake/video/e-fake...,fake
4,4,#NÃO É BEM ASSIM: Nem todo acompanhante de pas...,posts acertam na descrição de como obter desco...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fake


In [162]:
df['check'].value_counts(normalize=True)*100

check
fato    52.941176
fake    47.058824
Name: proportion, dtype: float64

In [163]:
print(df['titulo'][df['check'] == 'fato'])

1        Fato ou Fake estreia quadro no 'Mais Você'; as...
2        Paula Paiva apresenta o quadro "Fato ou Fake" ...
11       Fato ou Fake estreia quadro no 'Mais Você'; as...
12       Paula Paiva apresenta o quadro "Fato ou Fake" ...
21       Fato ou Fake estreia quadro no 'Mais Você'; as...
                               ...                        
16978    Câmara discute nos bastidores emendas e anisti...
16979    ANA FLOR: PEC da Blindagem é escárnio do Congr...
16980    SADI: Fazenda vira alvo de ministros em reuniã...
16981    Bolo que envenenou família foi feito de última...
16982    Rio pode ter recorde de calor hoje; veja as ca...
Name: titulo, Length: 8991, dtype: object
